# Further process slabs created in `create_slabs.ipynb`
---



# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import json

import pandas as pd

from tqdm.notebook import tqdm

import time
from pathlib import Path
import pickle


# #########################################################
from methods import (
    get_df_dft,
    # symmetrize_atoms,
    # get_structure_coord_df, remove_atoms,
    )

from local_methods import (
    constrain_slab,
    resize_z_slab,
    )

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/creating_slabs


# Read `df_slab` dataframe

In [2]:
path_i = os.path.join("out_data", "df_slab.pickle")
if Path(path_i).is_file():
    with open(path_i, "rb") as fle:
        df_slab = pickle.load(fle)
        print("df_slab_old.shape:", df_slab.shape)
else:
    print("Oh oh, file should be here")

df_slab_old.shape: (432, 5)


In [3]:
df_dft = get_df_dft()

In [4]:
if "status" in df_slab.columns:
    print("DSKFJKS")
    df_slab_1 = df_slab[df_slab.status != "Took too long"]
else:
    print("eh")
    df_slab_1 = df_slab

DSKFJKS


In [5]:
# df_slab_1.status.unique()

# # df_slab[df_slab.status != "Took too long"]

# df_slab.columns.

In [6]:
directory = "out_data/final_slabs_1"
if not os.path.exists(directory):
    os.makedirs(directory)

directory = "out_data/final_slabs_2"
if not os.path.exists(directory):
    os.makedirs(directory)

directory = "out_data/bulk_structures_temp"
if not os.path.exists(directory):
    os.makedirs(directory)

In [7]:
data_dict_list = []
iterator = tqdm(df_slab_1.index.tolist(), desc="1st loop")

# for i_cnt, row_i in df_slab_1.iterrows():
for i_cnt, slab_id_i in enumerate(iterator):
    row_i = df_slab_1.loc[slab_id_i]

    data_dict_i = dict()
    t0 = time.time()

    # #####################################################
    slab_id = row_i.name
    slab = row_i.slab_final
    bulk_id_i = row_i.bulk_id
    facet_i = row_i.facet
    # #####################################################

    # slab_constrained = constrain_slab(atoms=slab)
    slab_final = resize_z_slab(atoms=slab, vacuum=15)
    slab_final.center()
    slab_final.wrap()

    # #####################################################
    data_dict_i["slab_id"] = slab_id
    data_dict_i["bulk_id"] = bulk_id_i
    data_dict_i["facet"] = facet_i
    data_dict_i["slab_final"] = slab_final
    # data_dict_i["slab_constrained"] = slab_constrained
    data_dict_i["loop_time"] = t0 - time.time()
    
    # print(data_dict_i["loop_time"])
    data_dict_list.append(data_dict_i)
    
    file_name = row_i.bulk_id + "__" + row_i.name + "__" + row_i.facet + ".cif"
    slab_final.write("out_data/final_slabs_1/" + file_name)

    file_name = row_i.bulk_id + "__" + row_i.name + "__" + row_i.facet + ".traj"
    slab_final.write("out_data/final_slabs_1/" + file_name)

In [8]:
df = pd.DataFrame(data_dict_list)

df.head()

,slab_id,bulk_id,facet,slab_final,loop_time
0,hirapewu_45,73nhvjxyxf,011,"(Atom('O', [4.977352182850001, 4.9964319, 7.50...",-0.024913
1,hodukuma_46,73nhvjxyxf,110,"(Atom('O', [3.57473825025, 0.5706587562499995,...",-0.002285
2,tetekuse_50,73nhvjxyxf,010,"(Atom('O', [3.133508074350001, 1.6536866610000...",-0.002622
3,kurapadu_87,73nhvjxyxf,100,"(Atom('O', [0.7876549999999998, 8.564576479249...",-0.001898
4,rutofupe_76,73nhvjxyxf,001,"(Atom('O', [0.7876549999999998, 3.12983281715,...",-0.001814


In [9]:
# #######################################################################
data_path = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/creating_slabs/selecting_bulks",
    "out_data/data.json")
with open(data_path, "r") as fle:
    data = json.load(fle)
# #######################################################################

bulk_ids__octa_unique = data["bulk_ids__octa_unique"]

In [10]:
df_i = df[df.bulk_id.isin(bulk_ids__octa_unique)]

for i_cnt, row_i in df_i.iterrows():
    slab = row_i.slab_final

    file_name = row_i.bulk_id + "__" + row_i.slab_id + "__" + row_i.facet + ".cif"
    slab.write("out_data/final_slabs_2/" + file_name)

    file_name = row_i.bulk_id + "__" + row_i.slab_id + "__" + row_i.facet + ".traj"
    slab.write("out_data/final_slabs_2/" + file_name)

In [47]:
partial = "gisasa"
for i_cnt, row_i in df_i.iterrows():

    bulk_id = row_i.bulk_id
    slab_id = row_i.slab_id

    if partial in slab_id:
        print("slab:", slab_id)
        print("bulk:", bulk_id)

# slab_id

slab: gisasaho_61
bulk: 6qve7u7fcu


In [12]:
# partial = "zy7ac"
# partial = "vk9"
partial = "x19jv"
for i in df_dft.index.tolist():
    if partial in i:
        print(i)

x19jvf73mh


In [13]:
# for i_cnt, row_i in df_dft.loc[bulk_ids__octa_unique].iterrows():
#     atoms = row_i.atoms
    
#     atoms.write("out_data/bulk_structures_temp/" + row_i.name + ".cif")

In [14]:
# bulks_to_not_process = [
#     "vqmdms9e8u",
#     "6f9fzy7ac5",
#     "8jvfcyvk92",
#     "8s9ecomu9g",
#     "binlzubqnp",
#     "c38dvjmwbu",
#     "cf9lbhvjmq",
#     "ckbg7d9u6u",
#     "nk8in1n383",
#     "nkm5bynaza",
#     "nt6f6ubucs",
#     "x19jvf73mh",
#     ]

# # #########################################################
# import json
# data_path = os.path.join("in_data/bulks_to_not_run.json")
# with open(data_path, "w") as fle:
#     json.dump(bulks_to_not_process, fle, indent=2)
# # #########################################################

In [15]:
# import copy

# import numpy as np
# import pandas as pd

# from ase import io

# # from tqdm import tqdm
# from tqdm.notebook import tqdm

# # #########################################################


# # #########################################################
# from misc_modules.pandas_methods import drop_columns
# from misc_modules.misc_methods import GetFriendlyID
# from ase_modules.ase_methods import view_in_vesta

# # #########################################################
# from methods import (
#     get_df_dft, symmetrize_atoms,
#     get_structure_coord_df, remove_atoms)
# from proj_data import metal_atom_symbol

# # #########################################################
# from local_methods import (
#     analyse_local_coord_env, check_if_sys_processed,
#     remove_nonsaturated_surface_metal_atoms,
#     remove_noncoord_oxygens,
#     create_slab_from_bulk,
#     get_slab_thickness,
#     remove_highest_metal_atoms,
#     remove_all_atoms_above_cutoff,
#     create_final_slab_master,
#     )

# from local_methods import calc_surface_area